https://www.kaggle.com/code/ikeppyo/jpx-lightgbm-demo
をfork。

データの詳細は https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/data を参照  
時系列APIの使い方は https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/overview/evaluation を参照  
オプションコード体系の日本語版は https://www.jpx.co.jp/sicc/securities-code/nlsgeu0000032d48-att/(HP)sakimono20220208.pdf を参照  
証券コード関係は https://www.jpx.co.jp/sicc/securities-code/01.html を参照

# 準備
* 使用するライブラリをインポートします

In [ ]:
import os
import gc
from pathlib import Path
from decimal import ROUND_HALF_UP, Decimal

import pandas as pd
import numpy as np
import datetime
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: #numericsのデータ型の範囲内のときに処理を実行. データの最大最小値を元にデータ型を効率的なものに変更
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int8).min) & (c_max < np.iinfo(np.int8).max):
                    df[col] = df[col].astype(np.int8)
                elif (c_min > np.iinfo(np.int16).min) & (c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min) & (c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min) & (c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)  
            else:
                if (c_min > np.finfo(np.float16).min) & (c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min) & (c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# データ読み込み

以下のファイル群を読み込みます。
* stock_list.csv
* train_files 配下のファイル（Dateの範囲は2017-01-04 ～ 2021-12-03）
* supplemental_files 配下のファイル（Dateの範囲は2021-12-06 ～ 2022-02-28 ※2022/04/05現在）

## Function

In [ ]:
def read_files(dir_name: str = 'train_files'):
    base_path = Path(f'../input/jpx-tokyo-stock-exchange-prediction/{dir_name}')
    prices = pd.read_csv(base_path / 'stock_prices.csv')
    options = pd.read_csv(base_path / 'options.csv')
    financials = pd.read_csv(base_path / 'financials.csv')
    trades = pd.read_csv(base_path / 'trades.csv')
    secondary_prices = pd.read_csv(base_path / 'secondary_stock_prices.csv')
    
#     prices = reduce_mem_usage(prices)
#     options = reduce_mem_usage(options)
#     financials = reduce_mem_usage(financials)
#     trades = reduce_mem_usage(trades)
#     secondary_prices = reduce_mem_usage(secondary_prices)
    
    return prices, options, financials, trades, secondary_prices

## Exec

In [ ]:
%%time

stock_list = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')
train_files = read_files('train_files')
supplemental_files = read_files('supplemental_files')

# データ統合

* 読み込んだデータを結合して、一つのファイルに纏めます。

## Function

`merge_data`関数では各ファイルを水平方向に結合します。現時点では`stock_prices`、`stock_list`しか使っていません。  
コメントアウトを解除すれば`trades`、`financials`とも結合は可能ですが、  
これらのデータは、有効なレコードが発生するタイミングが日次ではないため、学習データとして意味のあるものとするためには「直近に発生した有効なレコードの値を引き継ぐ」などの対処が必要となります。  
`options`に関しては、[OptionsCodeの附番規則](https://www.jpx.co.jp/sicc/securities-code/nlsgeu0000032d48-att/(HP)sakimono20220208.pdf)を見ていけば適切な使い方が見えそうです。

In [ ]:
def merge_data(prices, options, financials, trades, secondary_prices, stock_list):
    # stock_prices がベース
    base_df = prices.copy()
    
    # stock_listと結合
    _stock_list = stock_list.copy()
    _stock_list.rename(columns={'Close': 'Close_x'}, inplace=True)
    base_df = base_df.merge(_stock_list, on='SecuritiesCode', how="left")

    # tradesと結合
    # stock_listのNewMarketSegmentと紐づくよう、tradesのSection項目を編集する
    # _trades = trades.copy()
    # _trades['NewMarketSegment'] = _trades['Section'].str.split(' \(', expand=True)[0]
    # base_df = base_df.merge(_trades, on=['Date', 'NewMarketSegment'], how="left")

#     # financialsと結合
#     _financials = financials.copy()
#     _financials.rename(columns={'Date': 'Date_x', 'SecuritiesCode': 'SecuritiesCode_x'}, inplace=True)
#     use_cols = ['DateCode',
#                'NetSales',
#                'OperatingProfit',
#                'OrdinaryProfit',
#                'Profit', 
#                'EarningsPerShare', 
#                'TotalAssets', 
#                'Equity', 
#                'EquityToAssetRatio', 
#                'NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock']
#     # メモリがカツカツなのでカラムを使うものだけに絞る
#     _financials = _financials[use_cols]
#     _financials.replace(['-', '－'], np.nan, inplace=True)
#     for col in use_cols[1:]:
#         _financials[col] = _financials[col].astype(float)
#     prev_column = set(_financials.columns)
    
#     # 同じDateCodeで複数行あるのでまとめあげる
#     _financials = _financials.groupby('DateCode').max().reset_index()
#     assert prev_column == set(_financials.columns.to_list())
#     _financials.drop_duplicates(subset='DateCode', inplace=True)
#     assert prev_column == set(_financials.columns.to_list())
#     base_df = base_df.merge(_financials, left_on='RowId', right_on='DateCode', how="left")
    
    return base_df

`adjust_price`関数は[Train Demo](https://www.kaggle.com/code/smeitoma/train-demo)で紹介されている関数をほぼそのまま使わせて頂いています。（Dateのインデックス化だけコメントアウトしています）  
項目追加が発生するため、統合の範囲を超えていますが、関数内でソートやインデックスの生成といった操作が行われるため、この段階で実行しています。  
  
関数では**AdjustedClose**という項目が生成されます。  
株式は、分割や併合によって株価が大きく変動することがありますが、**Close**の代わりに**AdjustedClose**を使うことで、この影響を減少させることができるとのことです。

In [ ]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    # price.set_index("Date", inplace=True)
    return price

In [ ]:
def collector(prices, options, financials, trades, secondary_prices, stock_list):
    # 読み込んだデータを統合して一つのファイルに纏める
    base_df = merge_data(prices, options, financials, trades, secondary_prices, stock_list)
    
    # AdjustedClose項目の生成
    base_df = adjust_price(base_df)
    
    return base_df

## Exec

supplemental filesを使わない場合は4行目以降をコメントアウトします。

In [ ]:
%%time

base_df = collector(*train_files, stock_list)
supplemental_df = collector(*supplemental_files, stock_list)
base_df = pd.concat([base_df, supplemental_df]).reset_index(drop=True)

# 東証が障害で止まった日は除外
base_df = base_df[base_df['Date'] != '2020-10-01']

# base_df['Target'] = base_df['Target'] * 100

In [ ]:
gc.collect()

In [ ]:
base_df

In [ ]:
def add_rank_with_new_col_name(df, col_name="pred", new_col_name='Rank', ascend=False):
    df[new_col_name] = df.groupby("Date")[col_name].rank(ascending=ascend, method="first") - 1 
    df[new_col_name] = df[new_col_name].astype("int")
    return df

# https://arxiv.org/abs/2012.07245 の実装

# spectral residualの抽出

In [ ]:
def generate_return_df(base_df):
    """
    欠損値は0で埋める
    """
    tmp = base_df[["RowId", "Date", "SecuritiesCode", "AdjustedClose"]]
    tmp = tmp.sort_values(['Date', 'SecuritiesCode'])
    tmp["return"] = tmp.groupby("SecuritiesCode")["AdjustedClose"].pct_change()
    tmp['return'].fillna(0, inplace=True)
    
    ret = tmp.pivot(index='Date', columns='SecuritiesCode', values='return')
    ret.fillna(0, inplace=True)
    return ret

def generate_log_return_df(base_df):
    """
    欠損値は0で埋める
    """
    tmp = base_df[["RowId", "Date", "SecuritiesCode", "AdjustedClose"]]
    tmp = tmp.sort_values(['Date', 'SecuritiesCode'])
    tmp["log_return"] = tmp.groupby("SecuritiesCode")["AdjustedClose"].apply(np.log).diff()
    tmp['log_return'].fillna(0, inplace=True)
    
    ret = tmp.pivot(index='Date', columns='SecuritiesCode', values='log_return')
    ret.fillna(0, inplace=True)
    return ret

def generate_log_return_df_per_code(base_df):
    tmp = base_df[["RowId", "Date", "SecuritiesCode", "AdjustedClose", "33SectorCode"]]
    tmp = tmp.sort_values(['Date', 'SecuritiesCode'])
    tmp["log_return"] = tmp.groupby("SecuritiesCode")["AdjustedClose"].apply(np.log).diff()
    tmp['log_return'].fillna(0, inplace=True)
    
    ret_dict = dict()
    for sectorCode in tmp['33SectorCode'].unique():
        ret = tmp[tmp['33SectorCode'] == sectorCode].pivot(index='Date', columns='SecuritiesCode', values='log_return')
        ret.fillna(0, inplace=True)
        for seccode in ret.columns:
            if not np.all(ret[seccode][-20:] != 0):
                print('drop', seccode)
                ret.drop(seccode, axis=1, inplace=True)
        ret_dict[sectorCode] = ret

    return ret_dict

In [ ]:
return_df = generate_return_df(base_df)
return_df.head()
log_return_df = generate_log_return_df(base_df)
log_return_per_code_df = generate_log_return_df_per_code(base_df)

In [ ]:
# 5月27日の段階で市場から撤退している銘柄がある。
# おそらく、1413, 4699, 8806
# mask = return_df.loc["2022-05-27"].isnull()
# print(return_df.columns[mask])


# generate_residual_spectral
$$
f(\mathbf{[\mathbf{r_{t-H+1}}, \cdots, \mathbf{r_{t}}}]):\mathbb{R}^{N\times H} \rightarrow \mathbb{R}^{N\times H}
$$

In [ ]:
# residual_spectralを生成する
# N×H行列
def generate_residual_spectral(return_df, current_date):
    horizon = 64
    num_remove_factors = 3
    date_idx_map = {}
    for i, date in enumerate(return_df.index):
        date_idx_map[date] = i
    idx = date_idx_map[current_date]
    assert idx > horizon
    # 直近256日分のデータをとってくる。
    left_idx = idx-horizon+1
    X = return_df[return_df.index[left_idx]:return_df.index[idx]]
    
    # 列方向を時間軸にする
    X = X.transpose()
    # 中心化
    X_tilde = X.apply(lambda x: x - x.mean(), axis=1)
    v, s, uh = np.linalg.svd(X_tilde.fillna(0).to_numpy(), full_matrices=False, compute_uv=True)

    for i in range(len(s)):
        s[i] = 0 if i < num_remove_factors else 1
    A = v @ np.diag(s)  @ v.T
    ret = pd.DataFrame(index=X_tilde.index, columns=X_tilde.columns)
    for date in return_df.index[idx-horizon+1:idx+1]:
        ret[date] = A @ X[date]
        
    for code in ret.index:
        ret.loc[code] = ret.loc[code] / np.linalg.norm(ret.loc[code].to_numpy())
    return ret

# model

modelを写像$\psi(\mathbf{x})$とすると、
$$
\psi(\mathbf{x}): \mathbb{R}^H \rightarrow \mathbb{R}^{Q-1} \\
\psi(\mathbf{x}) = \psi_2\left(\frac{1}{L}\sum_{i=1}^{L}{\psi_1(Resample(\mathbf{x}, \tau_i}))\right)\\
\psi_1: \mathbb{R}^{H'} \rightarrow \mathbb{R}^{K}, where\ K=256\\
\psi_2: \mathbb{R}^{K} \rightarrow \mathbb{R}^{Q-1}
$$

## distributional prediction
銘柄ごとの直近H日間の残差リターン$\tilde{\epsilon}_{i, t-H}, \tilde{\epsilon}_{i, t-H+1}, \cdots, \tilde{\epsilon}_{i, t-1}$が与えられたときに、1つ後の残差リターンの分布を予測する。  
すなわち、考えるのは以下の条件付き分布である。

$$
p(\tilde{\epsilon}_{i, t}|\tilde{\epsilon}_{i, t-H}, \tilde{\epsilon}_{i, t-H+1}, \cdots, \tilde{\epsilon}_{i, t-1})
$$

これをノンパラメトリックに推定するために、32分位点回帰を行って、各分位点の予測値を出すことで分布の平均と分散を求める。

今回の問題設定の場合は、2つ後の残差リターンを予測することになる。

$$
p(\tilde{\epsilon}_{i, t+2}|\tilde{\epsilon}_{i, t-H+1}, \tilde{\epsilon}_{i, t-H+1}, \cdots, \tilde{\epsilon}_{i, t})
$$

## Network Architectures

### Volatility invariance
ボラティリティクラスタリングは、ボラティリティについてのスケール不変性と考えられる。これをモデルに組み込むために、モデルは正の同次な写像であるとする。

### Fractal structure
株価はフラクタル性を持っていることで知られるため、異なる長さの区間のデータを取ってきて、同じ幅になるようにリサンプリングし、同じネットワークに流す。この際、時間軸のスケールファクターでリスケーリングを行う。このファクターはHurst指数が株価の場合はおよそ0.5であることが観測からわかっているため、$\tau^{0.5}$で割る。

### resample
フラクタルネットワークに流すためにリサンプリングを行う関数。これは一つの銘柄のH期間のデータを入力とし、固定長H'のベクトルを返す。
$$
Resample(x, \tau): \mathbb{R}^{H} \rightarrow \mathbb{R}^{H'}\\
\tau: scale\ parameter
$$

この関数の処理は4段階からなっている。

1. 累積和
1. Resample
1. 一階差分
1. Rescale

また、このResample関数は正の同次性を持っている。


In [ ]:
import math
def resample(x, tau, sample_size=64):
    ret = []
    for i in range(len(x)):
        row = x[i]
        z = []
        s = 0
        for xx in row:
            s += xx
            z.append(s)
        z_tau = z[-math.ceil(len(z)*tau):]

        resampled_z = []
        pos = 0
        while pos < len(z_tau):
            resampled_z.append(z_tau[int(pos)])
            pos += len(z_tau) / sample_size

        diff = [resampled_z[0]] + [resampled_z[i+1] - resampled_z[i] for i in range(len(resampled_z)-1)]
        diff = np.array(diff)
        diff *= tau**(-1/2)

        ret.append(diff)
        assert len(diff) == sample_size
    ret = np.array(ret)
    assert np.isnan(ret).any() == False

    return ret

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
class FractalNetwork(nn.Module):
    
    def __init__(self, dim_input=64, n_hidden=256, Q=32, device=None):
        super().__init__()
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device
        self.dim_input = dim_input
        self.n_hidden = n_hidden
        self.Q = Q
        self.psi1 = Psi1(dim_input=dim_input, K=n_hidden)
        self.psi2 = Psi2(dim_input=n_hidden, Q=Q)
        
    def forward(self, inputs):
        psi1_result = torch.zeros((inputs.shape[0], 256), device=torch.device(self.device))
#         psi1_result.to(self.device)
#         print('psi1_result device:', psi1_result.device)
        L = 21
        for tau in [4**(-j/(L-1)) for j in range(L)]:
            resampled_inputs = resample(inputs, tau, self.dim_input)
            resampled_inputs = torch.from_numpy(resampled_inputs.astype(np.float32))
            resampled_inputs = resampled_inputs.to(self.device)
#             print('resampled_inputs device:', resampled_inputs.device)
#             tmp = self.psi1(resampled_inputs)
#             print('debug:psi1_result', tmp)
#             print(tmp.shape)
#             print('psi1.device: ',self.psi1.device)
            psi1_result += self.psi1(resampled_inputs)
        psi1_result /= L
        
        return self.psi2(psi1_result)
    
class Psi1(nn.Module):
    
    def __init__(self, device=None, dim_input=64, K=256):
        super().__init__()
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device
            
        self.layers = nn.Sequential(
            torch.nn.Linear(dim_input, 256, bias=False),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 256, bias=False),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 256, bias=False),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, K, bias=False)
            )
        
    def forward(self, inputs):
        return self.layers(inputs)
    
class Psi2(nn.Module):
      
    def __init__(self, device=None, dim_input=256, Q=32):
        super().__init__()
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        self.layers = nn.Sequential(
            torch.nn.Linear(dim_input, 128, bias=False), #1
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 128, bias=False), #2
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 128, bias=False), #3
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 128, bias=False), #4
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 128, bias=False), #5
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 128, bias=False), #6
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 128, bias=False), #7
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 128, bias=False), #8
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, Q-1, bias=False)
        )


    def forward(self, inputs):
        return self.layers(inputs)
            
        
        

In [ ]:
# Loss function
class QuantileLoss(nn.Module):
    def __init__(self, Q=32):
        super(QuantileLoss, self).__init__()
        self.Q = Q
        
    def forward(self, outputs, targets, device):
        batch_loss = torch.zeros((outputs.shape[0], 1), device=torch.device(device))
        for i in range(outputs.shape[0]):
            assert outputs.shape[1] == self.Q-1
            output = outputs[i]
            losses = torch.zeros(self.Q-1, device=torch.device(device))

            for j in range(self.Q - 1):
                alpha = (j+1) / self.Q
                diff = targets[i] - output[j]
                losses[j] = torch.max((alpha-1)*diff, alpha*diff)
            batch_loss[i] = (torch.sum(losses))
#         batch_loss = torch.cat(batch_loss, dim=1)
        return torch.mean(batch_loss)
            
        

In [ ]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    
    def __init__(self, residual_spectral, targets):
        super().__init__()
        self.residual_spectral = residual_spectral
        self.targets = targets
        self.idx_map = dict()
        self.inverse_idx_map = dict()
        for i, code in enumerate(residual_spectral.index):
            self.idx_map[code] = i
            self.inverse_idx_map[i] = code
    
    def __getitem__(self, idx):
        inputs = self.residual_spectral.loc[self.inverse_idx_map[idx]].values.astype(np.float32)
        target = self.targets.loc[self.inverse_idx_map[idx]].astype(np.float32)
        return inputs, target
    
    def __len__(self):
        return len(self.residual_spectral)

In [ ]:
gc.collect()

# calculate the weights of portfolio from the prediction of quantiles of distribution
modelの出力値である分位点の予測値からポートフォリオのウェイトを計算する。
submitの形式にする場合は、事前にdataframeを作っておき、銘柄のカラムを追加、Dateのカラムは予測日とする。

その後、calc_weightsの返り値をpredカラムの値とし、その値からadd_rankで順位付けを行う。


In [ ]:
def get_mean_and_variance(outputs):
    means = torch.mean(outputs.detach(), dim=1, keepdim=True)
    variances = torch.var(outputs.detach(), dim=1, keepdim=True)
    return means, variances

def calc_weights(means, variances):
    ret = means / variances
    ret = ret.cpu()
    return ret.numpy()



In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

# train
今は2000銘柄のデータを同時に入力している。

batch_size=2000

のイメージ

もしかしたらもっと細かく分割したほうがいいかもしれない。その場合は、データを作成する際にうまくreshapeする。DataLoaderを使ってもいいかもしれない。

また、validationも学習した日の次の日のデータで行っているが、235日分のデータはtrainで使用したものと同じなので、もっと離したほうがいいかもしれない。

## version4
予測させようとするとすべてのデータに対して同じ出力になってしまった。  
なので、batch_sizeを小さくしてみる。

## version5
batch_size=10
効果なし

## version6
学習が進むにつれ同じ値になっていくことがわかった。  
また、一日ずれた日付の予測も同じになる。  
長期のhorizonでやっているためかもしれないので、horizonを短くしてやってみる。  
これを変更するにあたって、除く特異ベクトルの数、resampleの数なども変える必要がある。

In [ ]:
import random
def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True


torch_fix_seed()

In [ ]:
# SectorCodeごとの学習
epochs = 4
model = FractalNetwork(dim_input=32)
device = model.device
criterion = QuantileLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.train()
model.to(device)
print(model)

train_end_date = pd.to_datetime('2022-02-01')

for epoch in range(epochs):
    losses = []
    valid_losses = []
    for i, date in tqdm(list(enumerate(log_return_df.index[:-43]))[-360:-60:10]):
        if i <= 256 or date < pd.to_datetime('2020-12-23'):
            continue
        if date > train_end_date:
            break
        print('train date:', date)
        preds_df = pd.DataFrame()
        for code, log_return_df in log_return_per_code_df.items():
            target_date = log_return_df.index[i+2]
            residual_spectral = generate_residual_spectral(log_return_df, target_date)


            valid_target_dates = log_return_df.index[-50:-40]
            valid_residual_spectrals = [generate_residual_spectral(log_return_df, valid_date) for valid_date in valid_target_dates]

            train_dataset = MyDataset(residual_spectral, log_return_df.loc[target_date])
    #         valid_dataset = MyDataset(valid_residual_spectral, return_df.loc[valid_target_date])

            train_loader = DataLoader(train_dataset, batch_size=len(log_return_df.columns), shuffle=True)
    #         valid_loader = DataLoader(valid_dataset, batch_size=len(return_df.columns))
            batch_train_loss = 0
            batch_valid_loss = 0
            for j, (train_inputs, train_targets) in enumerate(train_loader):
                model.train()
                outputs = model(train_inputs)
                outputs = outputs.to(device)
                loss = criterion(outputs, train_targets, device)
                loss = loss.to(device)
                batch_train_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()

                optimizer.step()
            losses.append(batch_train_loss/(j+1))

            with torch.no_grad():
                model.eval()
                
                for ind, valid_target_date in enumerate(valid_target_dates):
                    valid_residual_spectral = valid_residual_spectrals[ind]
                    valid_dataset = MyDataset(valid_residual_spectral, log_return_df.loc[valid_target_date])
                    valid_loader = DataLoader(valid_dataset, batch_size=len(log_return_df.columns))
                    for k, (valid_inputs, valid_targets) in enumerate(valid_loader):
                        valid_outputs = model(valid_inputs)
                        valid_loss = criterion(valid_outputs, valid_targets, device)
                        batch_valid_loss += valid_loss.item()
                    means, variance = get_mean_and_variance(valid_outputs)
                    weights = calc_weights(means, variance)
    #                 print(weights)
                    pred_df = pd.DataFrame()
                    pred_df['SecuritiesCode'] = log_return_df.columns
                    pred_df['Date'] = valid_target_date
                    pred_df = pred_df.merge(base_df[['Date', 'SecuritiesCode', 'Target']], on=['Date','SecuritiesCode'],how='left')
#                     pred_df['Target'] = log_return_df.loc[valid_target_date].values
                    pred_df['pred'] = weights.squeeze()
                    pred_df.fillna(0, inplace=True)
                    preds_df = pd.concat([preds_df, pred_df])
                    preds_df = preds_df.reset_index(drop=True)
        preds_df = add_rank_with_new_col_name(preds_df, col_name='pred', new_col_name='Rank')
        preds_df = add_rank_with_new_col_name(preds_df, col_name='Target', new_col_name='TrueRank')
        sns.scatterplot(data=preds_df, x='pred', y='Target', hue='Date', alpha=0.5)
        fig = plt.figure(figsize=(10, 20))
        plt.show()
        print(preds_df)
        score = calc_spread_return_sharpe(preds_df)
        print('score:', score)

        valid_losses.append(batch_valid_loss/(k+1))
        print(f'{i+1}\t loss: ', losses[-1], '\t', 'valid_loss: ', valid_losses[-1])
#         if i %10 == 9:
#             print(f'{i+1}\t loss: ', losses[-1], '\t', 'valid_loss: ', valid_losses[-1])
        
            
            
    plt.figure()
    plt.plot(range(1, len(losses)+1), losses, label='train_loss')
    plt.plot(range(1, len(valid_losses)+1), valid_losses, label='valid_loss')
    plt.xlabel('iteration')
    plt.legend()
    plt.show()
        

In [ ]:
epochs = 1
batch_size = 5
model = FractalNetwork(dim_input=32)
device = model.device
criterion = QuantileLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.train()
model.to(device)
print(model)

train_end_date = pd.to_datetime('2022-02-01')

for epoch in range(epochs):
    losses = []
    valid_losses = []
    for i, date in tqdm(list(enumerate(log_return_df.index[:-43]))[-360:-60:10]):
        if i <= 256 or date < pd.to_datetime('2020-12-23'):
            continue
        if date > train_end_date:
            break
        print('train date:', date)
        target_date = log_return_df.index[i+2]
        residual_spectral = generate_residual_spectral(log_return_df, target_date)
        
        
        valid_target_dates = log_return_df.index[-50:-40]
        valid_residual_spectrals = [generate_residual_spectral(log_return_df, valid_date) for valid_date in valid_target_dates]
        
        train_dataset = MyDataset(residual_spectral, log_return_df.loc[target_date])
#         valid_dataset = MyDataset(valid_residual_spectral, return_df.loc[valid_target_date])
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#         valid_loader = DataLoader(valid_dataset, batch_size=len(return_df.columns))
        batch_train_loss = 0
        batch_valid_loss = 0
        for j, (train_inputs, train_targets) in enumerate(train_loader):
            model.train()
            outputs = model(train_inputs)
            outputs = outputs.to(device)
            loss = criterion(outputs, train_targets, device)
            loss = loss.to(device)
            batch_train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
        losses.append(batch_train_loss/(j+1))
        
        with torch.no_grad():
            model.eval()
            preds_df = pd.DataFrame()
            for ind, valid_target_date in enumerate(valid_target_dates):
                valid_residual_spectral = valid_residual_spectrals[ind]
                valid_dataset = MyDataset(valid_residual_spectral, log_return_df.loc[valid_target_date])
                valid_loader = DataLoader(valid_dataset, batch_size=len(log_return_df.columns))
                for k, (valid_inputs, valid_targets) in enumerate(valid_loader):
                    valid_outputs = model(valid_inputs)
                    valid_loss = criterion(valid_outputs, valid_targets, device)
                    batch_valid_loss += valid_loss.item()
                print('debug')
                print(valid_outputs)
                means, variance = get_mean_and_variance(valid_outputs)
                weights = calc_weights(means, variance)
                print(means, variance)
#                 print(weights)
                pred_df = pd.DataFrame()
                pred_df['SecuritiesCode'] = log_return_df.columns
                pred_df['Date'] = valid_target_date
                pred_df['Target'] = return_df.loc[valid_target_date].values
                pred_df['pred'] = weights.squeeze()
                pred_df.fillna(0, inplace=True)
                pred_df = add_rank_with_new_col_name(pred_df, col_name='pred', new_col_name='Rank')
                pred_df = add_rank_with_new_col_name(pred_df, col_name='Target', new_col_name='TrueRank')
                preds_df = pd.concat([preds_df, pred_df])
                preds_df = preds_df.reset_index(drop=True)
                print(pred_df)
            sns.scatterplot(data=preds_df, x='pred', y='Target', hue='Date', alpha=0.7)
            plt.figure(figsize=(8, 12))
            plt.show()
            score = calc_spread_return_sharpe(preds_df)
            print('score:', score)
            
        valid_losses.append(batch_valid_loss/(k+1))
        print(f'{i+1}\t loss: ', losses[-1], '\t', 'valid_loss: ', valid_losses[-1])
#         if i %10 == 9:
#             print(f'{i+1}\t loss: ', losses[-1], '\t', 'valid_loss: ', valid_losses[-1])
        
            
            
    plt.figure()
    plt.plot(range(1, len(losses)+1), losses, label='train_loss')
    plt.plot(range(1, len(valid_losses)+1), valid_losses, label='valid_loss')
    plt.xlabel('iteration')
    plt.legend()
    plt.show()
        

In [ ]:
# save model
model_path = 'residual_model.pth'

torch.save(model.state_dict(), model_path)

# evaluator

`calc_spread_return_sharpe`関数は[Train Demo](https://www.kaggle.com/code/smeitoma/train-demo)で紹介されている関数をそのまま使わせて頂いています。  
推論した**Rank**と、正解の**Target**を含むデータフレームを渡すと、コンテストの評価方法に沿ったスコアを計算してくれます。

In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [ ]:
# 予測用のデータフレームと、予測結果をもとに、スコアを計算する関数
def evaluator(df, pred):
    df["pred"] = pred
    df = add_rank(df)
    score = calc_spread_return_sharpe(df)
    return score

# 提出

時系列APIを使って推論結果を登録します。  
特徴量として、移動平均等の過去データを参照する値を採用しているため、時系列APIから得られたデータをため込む仕組みを実装する必要があります。  
この仕組みに関しては、[Start-to-finish demo based on s-meitoma + tweaks](https://www.kaggle.com/code/lowellniles/start-to-finish-demo-based-on-s-meitoma-tweaks)を参考にさせていただきました。  
以下のコードでは`past_df`というデータフレームに履歴データをため込む実装になっています。

In [ ]:
# 時系列APIのロード
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
# supplemental filesを履歴データの初期状態としてセットアップ
past_df = base_df[base_df['Date'] >= '2020-12-23'].copy()
past_df = past_df.reset_index(drop=True)